In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import MinMaxScaler

from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [ ]:
Bikedf = pd.read_csv('cleaned_df1.csv', na_filter=False)
df.head()


In [ ]:
# Filter the DataFrame for rows where segment is "ppr" or "pr"
filtered_data = df[Model['segment'].isin(['Price-performance recommendation', 'performance recommendation'])]

# Check the top values in the product_variant column for the filtered data
top_product_variants = filtered_data['product_variant'].value_counts()
display(top_product_variants)



In [ ]:
Bikedf["Premium"] = Bikedf["product_variant"].apply(lambda x: 1 if "Premium" in x else 0)
Bikedf["Fahrrad- und E-Bike-Versicherung"] = Bikedf["product_variant"].apply(lambda x: 1 if "Fahrrad- und E-Bike-Versicherung" in x else 0)
Bikedf["Plus"] = Bikedf["product_variant"].apply(lambda x: 1 if "Plus" in x else 0)
Bikedf["Superior"] = Bikedf["product_variant"].apply(lambda x: 1 if "Superior" in x else 0)
Bikedf["Premium 1.0"] = Bikedf["product_variant"].apply(lambda x: 1 if "Premium 1.0" in x else 0)

In [ ]:
#Bikedf = Bikedf.drop('product_variant', axis=1)

#Bikedf.head()


# Split the Dataset into Features (X) and Target Variables (y)

In [ ]:
# Assuming 'Price' and 'Rating' are your target variables
X = Bikedf.drop(['price', 'rating'], axis=1)
y = Bikedf['price']
#y_rating = Bikedf['rating']


In [ ]:
#Split the Dataset into Training and Testing Sets for Price Prediction# Encode categorical variables using LabelEncoder
#label_encoder = LabelEncoder()
#X_encoded_label = X.apply(label_encoder.fit_transform)



In [ ]:
# Use OneHotEncoder for categorical variables with more than two categories
categories_to_onehot = ['assistance', 'criteria', 'segment', 'provider', 'location', 'month', 'cost_category']
onehot_encoder = OneHotEncoder(drop='first')
X_encoded_onehot = pd.concat([X_encoded_label.drop(categories_to_onehot, axis=1), pd.get_dummies(X_encoded_label[categories_to_onehot], drop_first=True)], axis=1)

In [ ]:
# Split the data with 80% for training and 20% for testing
X_train_price, X_test_price, y_train_price, y_test_price = train_test_split(X, y_price, test_size=0.2, random_state=42)


In [ ]:
X_train_num = X_train_price.select_dtypes(np.number)
X_test_num  = X_test_price.select_dtypes(np.number) 

X_train_cat = X_train_price[categories_to_onehot]
X_test_cat  = X_test_price[categories_to_onehot]

X_train_binary_df = X_train_price[["Premium","Fahrrad- und E-Bike-Versicherung", "Plus", "Superior", "Premium 1.0"]]
X_test_binary_df  = X_test_price[["Premium","Fahrrad- und E-Bike-Versicherung", "Plus", "Superior", "Premium 1.0"]]

In [ ]:
import pickle
import os

onehot_encoder = OneHotEncoder(drop='first')
onehot_encoder.fit(X_train_cat)

# Save the encoder with pickle in the "../encoders/" folder
encoder = onehot_encoder
folder_path = r"C:\Users\priya\Documents\IRONHACK\Final_Project\Encoder"
# Specify the filename for the saved encoder
encoder_path = os.path.join(r"C:\Users\priya\Documents\IRONHACK\Final_Project\Encoder", "onehot_encoder.pkl")
# Save the encoder using pickle
with open(encoder_path, 'wb') as file:
    pickle.dump(encoder, file)

X_train_cat_encoded = onehot_encoder.transform(X_train_cat).toarray()
X_test_cat_encoded  = onehot_encoder.transform(X_test_cat).toarray()

X_train_cat_df = pd.DataFrame(X_train_cat_encoded, columns=onehot_encoder.get_feature_names_out(), index=X_train_cat.index)
X_test_cat_df  = pd.DataFrame(X_test_cat_encoded,  columns=onehot_encoder.get_feature_names_out(), index=X_test_cat.index)

In [ ]:
pt_x = PowerTransformer()
pt_x.fit(X_train_num)

# Save the transfomer in the "../transfomers/" folder
trained_transformer_object = pt_x
# Specify the folder path
folder_path = r"C:\Users\priya\Documents\IRONHACK\Final_Project\Transformer"
# Specify the filename for the saved transformer
transformer_filename = "power_transformer.pkl"
# Specify the full path, including the filename, for saving the transformer
transformer_path = os.path.join(r"C:\Users\priya\Documents\IRONHACK\Final_Project\Transformer", "power_transformer.pkl")
# Save the transformer using pickle
with open(transformer_path, 'wb') as file:
    pickle.dump(trained_transformer_object, file)
    
X_train_num_trans = pt_x.transform(X_train_num)
X_test_num_trans  = pt_x.transform(X_test_num)

X_train_num_df = pd.DataFrame(X_train_num_trans, columns=X_train_num.columns, index=X_train_num.index)
X_test_num_df  = pd.DataFrame(X_test_num_trans,  columns=X_test_num.columns, index=X_test_num.index)

X_train_num_df.head()


In [ ]:
X_train_num_df.describe().T

In [ ]:
X_train_merged = pd.concat([X_train_num_df, X_train_binary_df, X_train_cat_df], axis=1)
X_test_merged  = pd.concat([X_test_num_df, X_test_binary_df, X_test_cat_df], axis=1)

display(X_train_merged.head())
display(X_test_merged.head())

In [ ]:
# creating scaler object
scaler = MinMaxScaler()
#scaler.fit(X_train)
# fit transform on train set
X_train_scaled = scaler.fit_transform(X_train_merged)
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train_merged.columns)

# Save the scaler with pickle in the "../scalers/" folder
folder_path = r"C:\Users\priya\Documents\IRONHACK\Final_Project\Scaler"
# Specify the filename for the saved scaler
scaler_path = os.path.join(r"C:\Users\priya\Documents\IRONHACK\Final_Project\Scaler", "minmax_scaler.pkl")
# Save the scaler using pickle
with open(scaler_path, 'wb') as file:
    pickle.dump(scaler, file)

# transforming also test set
X_test_scaled = scaler.transform(X_test_merged)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X_test_merged.columns)

#X_train_trans = scaler.transform(X_train)
#X_test_trans  = scaler.transform(X_test)

#X_train_trans_df = pd.DataFrame(X_train_trans, columns=X_train.columns, index=X_train.index)
#X_test_trans_df  = pd.DataFrame(X_test_trans,  columns=X_test.columns, index=X_test.index)



In [ ]:
display(X_train_scaled_df.head())
display(X_test_scaled_df.head())

In [ ]:
#X_train_trans_df.describe().T

In [ ]:
y_train = Bikedf['price']
y_test = Bikedf['price']
# Reshape the y_train to a 2D array
y_train = y_train.values.reshape(-1, 1)
y_test = y_test.values.reshape(-1, 1)

In [ ]:
from sklearn.preprocessing import PowerTransformer
import os
import pickle

pt_y = PowerTransformer()
pt_y.fit(y_train)
# Save the transformer in the "../transformers/" folder

# Specify the folder path
folder_path = "Transformer"

# Specify the full path, including the filename, for saving the transformer
transformer_path = os.path.join(folder_path, "power_transformer.pkl")

# Save the transformer using pickle
with open(transformer_path, 'wb') as file:
    pickle.dump(pt_y, file)
   
y_train_trans = pt_y.transform(y_train)
y_test_trans  = pt_y.transform(y_test)

y_train_trans_df = pd.DataFrame(y_train_trans)
y_test_trans_df  = pd.DataFrame(y_test_trans)

In [ ]:
y_train_trans_df.head()
y_test_trans_df.head()


In [ ]:
#print(X_train_scaled_df.shape, len(y_train))
#Match the Number of Samples:
y_train = y_train[:39461]

In [ ]:
# Predict price -> Numerical -> Regression.
# LinearRegressor(),  RandomForestRegressor(), GradientBoostingRegressor(), KNNRegressor()
lm = LinearRegression()
lm.fit(X_train_scaled_df, y_train)


# Save your model with pickle in the "../models/" folder.
folder_path = r"C:\Users\priya\Documents\IRONHACK\Final_Project\Models"
model_path = os.path.join(folder_path, "linear_regression_model.pkl")
# Save the model using pickle
with open(model_path, 'wb') as file:
    pickle.dump(lm, file)
    print("The new directory is created!")
    
# y_pred_train_lm_trans = lm.predict(X_train_trans_df)
# y_pred_test_lm_trans  = lm.predict(X_test_trans_df)
# y_pred_train_lm = pt_y.inverse_transform(y_pred_train_lm_trans)
# y_pred_test_lm  = pt_y.inverse_transform(y_pred_test_lm_trans)

In [ ]:
# using linear model to get predictions from train and test set

y_train_pred = lm.predict(X_train_scaled_df)
y_test_pred = lm.predict(X_test_scaled_df)


#print(type(y_train))
#print(type(X_train_scaled_df))




In [ ]:
#print(X_train_scaled_df.shape, len(y_train.shape))
y_train = Bikedf['price']
print(y_train.shape)

In [ ]:
# Ensure X_train and y_train have the same number of rows
X_train = X_train.head(y_train.shape[0])

# Ensure X_test and y_test have the same number of rows
X_test = X_test.head(y_test.shape[0])

In [ ]:
# Convert y_train and y_test to DataFrames
y_train_df = pd.DataFrame(y_train, columns=['price'])
y_test_df = pd.DataFrame(y_test, columns=['price'])

# creating a dictionary comparing train to set predictions
results = {
    "Set": ["Train"] * X_train.shape[0] + ["Test"] * X_test.shape[0],
    "Real": list(y_train_df['price'].head(X_train.shape[0])) + list(y_test_df['price'].head(X_test.shape[0])),
    "Predicted": list(y_train_pred) + list(y_test_pred),
}

results_df = pd.DataFrame(results)
results_df.head()

In [ ]:
# creating a dictionary comparing train to set predictions
results = {
    "Set": ["Train"] * X_train.shape[0] + ["Test"] * X_test.shape[0],
    "Real": list(y_train.head(X_train.shape[0])) + list(y_test.head(X_test.shape[0])),
    "Predicted": list(y_train_pred) + list(y_test_pred),
}

results_df = pd.DataFrame(results)
results_df.head()

# Train the initial model on the training set for price prediction
initial_model_price = RandomForestRegressor(random_state=42)
initial_model_price.fit(X_train_price, y_train_price)

In [ ]:
#Evaluate the Initial Model for Price Prediction
# Make predictions on the test set for price prediction
y_pred_price = initial_model_price.predict(X_test_price)

# Evaluate the initial model's performance for price prediction
initial_mse_price = mean_squared_error(y_test_price, y_pred_price)
print(f"Initial Model MSE for Price Prediction: {initial_mse_price}")

In [ ]:
# Create the model for price prediction with default hyperparameters
model_price = RandomForestRegressor(random_state=42)

# Fit the model to the training data
model_price.fit(X_train_price, y_train_price)

# Make predictions on the test set for price prediction
y_pred_price = model_price.predict(X_test_price)

# Evaluate the model's performance for price prediction
mse_price = mean_squared_error(y_test_price, y_pred_price)
print(f"Model MSE for Price Prediction: {mse_price}")


In [ ]:

# Define a smaller hyperparameter grid
param_dist_price = {
    'model__n_estimators': [50, 100],
    'model__max_depth': [None, 10],
    'model__min_samples_split': [2, 5],
    'model__min_samples_leaf': [1, 2]
}

# Use RandomizedSearchCV instead of GridSearchCV
random_search_price = RandomizedSearchCV(pipeline_price, param_distributions=param_dist_price,
                                          n_iter=5, cv=3, scoring='neg_mean_squared_error', n_jobs=-1, random_state=42)

random_search_price.fit(X_price, y_price)

# Get the best hyperparameters for price prediction
best_params_price_random = random_search_price.best_params_
print(f"Best Hyperparameters for Price Prediction: {best_params_price_random}")

In [ ]:
#Fine-Tune Hyperparameters using GridSearchCV for Price Prediction
# Define hyperparameter grid for price prediction
param_grid_price = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Create the model for price prediction
model_price = RandomForestRegressor(random_state=42)

# Grid search for hyperparameter tuning for price prediction
grid_search_price = GridSearchCV(model_price, param_grid_price, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search_price.fit(X_train_price, y_train_price)

# Get the best hyperparameters for price prediction
best_params_price = grid_search_price.best_params_
print(f"Best Hyperparameters for Price Prediction: {best_params_price}")

In [ ]:
random_search_price = RandomizedSearchCV(model_price, param_distributions=param_grid_price, n_iter=10, cv=5, scoring='neg_mean_squared_error', random_state=42, n_jobs=-1)
random_search_price.fit(X_train_price, y_train_price)


In [ ]:
#Train the Final Model with Best Hyperparameters for Price Prediction
# Use the best hyperparameters to train the final model for price prediction
final_model_price = RandomForestRegressor(**best_params_price)
final_model_price.fit(X_train_price, y_train_price)


In [ ]:
#Evaluate the Final Model for Price Prediction
# Make predictions on the test set using the final model for price prediction
y_pred_final_price = final_model_price.predict(X_test_price)

# Evaluate the final model's performance for price prediction
final_mse_price = mean_squared_error(y_test_price, y_pred_final_price)
print(f"Final Model MSE for Price Prediction: {final_mse_price}")


# Perform clustering analysis to group customers based on criteria like bike type, assistance, and segment. This can help in tailoring your products to specific customer needs.

In [ ]:
# Select relevant features for clustering (bike type, assistance, and segment)
features = ['criteria', 'assistance', 'segment']
X = Model[features]

# Handle missing values if any
X = X.dropna()

# Encode categorical variables if necessary
X_encoded = pd.get_dummies(X)

In [ ]:
# Standardize the data by scaling the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_encoded)

In [ ]:
# Use the Elbow method to find the optimal number of clusters
wcss = []
for k in range(1, 11):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X_scaled)
    wcss.append(kmeans.inertia_)

# Plot the within-cluster sum of squares (WCSS) for different values of k
plt.plot(range(1, 11), wcss)
plt.xlabel('Number of Clusters (k)')
plt.ylabel('WCSS')
plt.title('Elbow Method')
plt.show()

In [ ]:
# Select the desired number of clusters based on the Elbow method (for example, k=3)
kmeans = KMeans(n_clusters=3, random_state=42)
kmeans.fit(X_scaled)

# Add the predicted cluster labels to the original DataFrame
Model['cluster_label'] = kmeans.labels_

In [ ]:
# Scatter plot of data points colored by cluster labels
plt.scatter(X['criteria'], X['assistance'], c=Model['cluster_label'], cmap='viridis')
plt.xlabel('Bike Type')
plt.ylabel('Assistance')
plt.title('Customer Segmentation')

# Add a colorbar to indicate the cluster labels
cbar = plt.colorbar()
cbar.set_label('Cluster Labels')

# Customize the plot with additional options (if needed)
# For example, you can adjust the marker size, add gridlines, or modify the axis limits

plt.show()

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, r2_score
import pickle
import os

# Assuming you have X_train, y_train, X_test, y_test

# Initialize KNNRegressor
knn_regressor = KNeighborsRegressor(n_neighbors=5)

# Fit the model on the training data
knn_regressor.fit(X_train, y_train)

# Save the model with pickle
folder_path = "Models"
model_path = os.path.join(folder_path, "knn_regressor_model.pkl")

with open(model_path, 'wb') as file:
    pickle.dump(knn_regressor, file)

# Make predictions on the training set
y_train_pred = knn_regressor.predict(X_train)

# Make predictions on the test set
y_test_pred = knn_regressor.predict(X_test)

# Evaluate the model
train_rmse = mean_squared_error(y_train, y_train_pred, squared=False)
test_rmse = mean_squared_error(y_test, y_test_pred, squared=False)

train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)

print(f"Training RMSE: {train_rmse}")
print(f"Testing RMSE: {test_rmse}")
print(f"Training R^2: {train_r2}")
print(f"Testing R^2: {test_r2}")
